In [27]:
import pandas as pd
import json
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

# NEIGHBORHOOD DATA

In [42]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
table = pd.read_html(url)
table[0].rename({"Neighbourhood":"Neighborhood"},axis=1,inplace=True)
table[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [43]:
import numpy as np
table[0].replace("Not assigned", np.nan, inplace=True)
table[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [44]:
table[0].reset_index(drop=True, inplace=True)
table[0].head(10)

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,NaN,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


# NEIGHBORHOOD ANALYSIS

In [45]:
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal Code'))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(latitude,longitude )

43.648690000000045 -79.38543999999996


In [46]:
url = 'http://cocl.us/Geospatial_data'
geotable = pd.read_csv("Geospatial_Coordinates.csv")
geotable.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
toronto = pd.merge(table[0], geotable, left_on='Postal Code', right_on='Postal Code', left_index=False, right_index=False)
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [49]:
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario')
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

#create a folium map of Toronto with Boroughs
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill_color='blue').add_to(map_Toronto)
map_Toronto

In [50]:
downtown = toronto[toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [51]:
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal_Code'))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

map_Downtown = folium.Map(location=[latitude, longitude], zoom_start=13
                        )
for lat, lng, neighborhood in zip(downtown['Latitude'], downtown['Longitude'], downtown['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill_color='blue').add_to(map_Downtown)
map_Downtown

In [52]:
CLIENT_ID = 'Z1JHMU3FKCYUXJEMXDSPYSPMB0UGUPCS444ITU3X2OEEEOMC'
CLIENT_SECRET = 'POVV2BNFHCTBN1DMPSGL4IAY3Y0MNXNUCDIORFGUBTDT0EPO'
VERSION = '20180605'
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: Z1JHMU3FKCYUXJEMXDSPYSPMB0UGUPCS444ITU3X2OEEEOMC
CLIENT_SECRET: POVV2BNFHCTBN1DMPSGL4IAY3Y0MNXNUCDIORFGUBTDT0EPO


In [54]:
RegentHarbour = downtown

neighborhood_name = RegentHarbour.loc[0, 'Neighborhood']
neighborhood_latitude = RegentHarbour.loc[0, 'Latitude']
neighborhood_longitude = RegentHarbour.loc[0, 'Longitude']
print('The latitude and longitude of {} are {} and {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

The latitude and longitude of Regent Park, Harbourfront are 43.6542599 and -79.3606359.


In [55]:
LIMIT = 100
radius = 500
Top100 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
Top100

'https://api.foursquare.com/v2/venues/explore?client_id=Z1JHMU3FKCYUXJEMXDSPYSPMB0UGUPCS444ITU3X2OEEEOMC&client_secret=POVV2BNFHCTBN1DMPSGL4IAY3Y0MNXNUCDIORFGUBTDT0EPO&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=100'

In [56]:


results = requests.get(Top100).json()
results



{'meta': {'code': 200, 'requestId': '60535887cf533316bdde9329'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [57]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
            
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [58]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# Filter the columns that we'd like in the df.
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.address']
nearby_venues = nearby_venues.loc[:, filtered_columns]

#Filter the category for each row. (I feel like this could be considered applying our category definition to the df, how we'd like it displayed.)
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#Define how we'd like to separate the columns.
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-58-5e404d2f6f2a>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng,address
0,Roselle Desserts,Bakery,43.653447,-79.362017,362 King St E
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809,368 King St E
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008,461 Cherry St
3,Body Blitz Spa East,Spa,43.654735,-79.359874,497 King Street East
4,Impact Kitchen,Restaurant,43.656369,-79.356980,573 King St E


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
toronto.head()

45 venues were returned by Foursquare.


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [60]:
downtown.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng, in zip(names, latitudes, longitudes):
        print(name)
        #api request
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        #get request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #return only the info we want
        venues_list.append([(name, lat, lng,
                           v['venue']['name'],
                           v['venue']['location']['lat'],
                           v['venue']['location']['lng'],
                           v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood'],['Neighborhood Latitude'],['Neighborhood Longitude'],['Venue'],['Venue Latitude'],['Venue Longitude'],['Venue Category']
            
    return(nearby_venues)

In [62]:
downtown_venues = getNearbyVenues(names=downtown['Neighborhood'], latitudes=downtown['Latitude'], longitudes=downtown['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [63]:
print(downtown_venues.shape)
downtown_venues.head()

(1224, 7)


,"(Neighborhood,)","(Neighborhood Latitude,)","(Neighborhood Longitude,)","(Venue,)","(Venue Latitude,)","(Venue Longitude,)","(Venue Category,)"
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [64]:
list(downtown_venues.columns.values)

[('Neighborhood',),
 ('Neighborhood Latitude',),
 ('Neighborhood Longitude',),
 ('Venue',),
 ('Venue Latitude',),
 ('Venue Longitude',),
 ('Venue Category',)]

In [65]:
downtown_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [66]:
#How many venues are in each Neighborhood?
downtown_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [67]:
print('There are {} unique categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 210 unique categories.


In [69]:
downtown_grouped = downtown_venues.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
0,Berczy Park,43.644771,-79.373306,43.647112,-79.373668
1,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,43.631002,-79.395322
2,Central Bay Street,43.657952,-79.387383,43.658625,-79.386286
3,Christie,43.669542,-79.422564,43.670523,-79.423555
4,Church and Wellesley,43.665860,-79.383160,43.666439,-79.382997
5,"Commerce Court, Victoria Hotel",43.648199,-79.379817,43.648804,-79.379452
6,"First Canadian Place, Underground city",43.648429,-79.382280,43.648769,-79.382467
7,"Garden District, Ryerson",43.657162,-79.378937,43.656169,-79.380403
8,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,43.641462,-79.382480
9,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,43.653967,-79.401479


In [70]:
#Find the top 5 venues for each neighborhood.
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                    venue   freq
0          Venue Latitude  43.65
1   Neighborhood Latitude  43.64
2  Neighborhood Longitude -79.37
3         Venue Longitude -79.37


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                    venue   freq
0   Neighborhood Latitude  43.63
1          Venue Latitude  43.63
2  Neighborhood Longitude -79.39
3         Venue Longitude -79.40


----Central Bay Street----
                    venue   freq
0   Neighborhood Latitude  43.66
1          Venue Latitude  43.66
2  Neighborhood Longitude -79.39
3         Venue Longitude -79.39


----Christie----
                    venue   freq
0   Neighborhood Latitude  43.67
1          Venue Latitude  43.67
2  Neighborhood Longitude -79.42
3         Venue Longitude -79.42


----Church and Wellesley----
                    venue   freq
0   Neighborhood Latitude  43.67
1          Venue Latitude  43.67
2  Neighborhood Longitude

# CLUSTER NEIGHBORHOOD

In [72]:
toronto_buroughs = toronto[toronto['Borough'].astype(str).str.contains('Toronto')]
toronto_buroughs

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [73]:
# Let's get that list of venues in a df with the lat/long for the venue.
latitudes = toronto_buroughs['Latitude']
longitudes = toronto_buroughs['Longitude']

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng, in zip(names, latitudes, longitudes):
        print(name)
        #api request
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        #get request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #return only the info we want
        venues_list.append([(name, lat, lng,
                           v['venue']['name'],
                           v['venue']['location']['lat'],
                           v['venue']['location']['lng'],
                           v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood'],['Neighborhood Latitude'],['Neighborhood Longitude'],['Venue'],['Venue Latitude'],['Venue Longitude'],['Venue Category']
            
    return(nearby_venues)

In [74]:


toronto_buroughs_venues = getNearbyVenues(names=toronto_buroughs['Neighborhood'], latitudes=toronto_buroughs['Latitude'], longitudes=toronto_buroughs['Longitude'])



Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harb

In [81]:
toronto_buroughs_grouped = toronto_buroughs.groupby('Neighborhood').mean().reset_index()
toronto_buroughs_grouped

,Neighborhood,Latitude,Longitude
0,Berczy Park,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
2,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
4,Central Bay Street,43.657952,-79.387383
5,Christie,43.669542,-79.422564
6,Church and Wellesley,43.665860,-79.383160
7,"Commerce Court, Victoria Hotel",43.648198,-79.379817
8,Davisville,43.704324,-79.388790
9,Davisville North,43.712751,-79.390197


In [82]:
#Top 5 venue from each neighborhood.
num_top_venues = 5

for hood in toronto_buroughs_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_buroughs_grouped[toronto_buroughs_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
       venue   freq
0   Latitude  43.64
1  Longitude -79.37


----Brockton, Parkdale Village, Exhibition Place----
       venue   freq
0   Latitude  43.64
1  Longitude -79.43


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
       venue   freq
0   Latitude  43.66
1  Longitude -79.32


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
       venue   freq
0   Latitude  43.63
1  Longitude -79.39


----Central Bay Street----
       venue   freq
0   Latitude  43.66
1  Longitude -79.39


----Christie----
       venue   freq
0   Latitude  43.67
1  Longitude -79.42


----Church and Wellesley----
       venue   freq
0   Latitude  43.67
1  Longitude -79.38


----Commerce Court, Victoria Hotel----
       venue   freq
0   Latitude  43.65
1  Longitude -79.38


----Davisville----
       venue   freq
0   Latitude  43.70
1  Longitude -79.39


----Davisville North----
     

In [84]:
kclusters = 5
tb_grouped_clustering = toronto_buroughs_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tb_grouped_clustering)
kmeans.labels_[0:10]

array([1, 4, 3, 1, 1, 4, 1, 1, 0, 0])

In [85]:
#Create a new df that includes the cluster and Top 10 for each neighborhood
#add clustering labels
Tburoughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
TB_merged = toronto_buroughs

#merge TB_merged w/ toronto to add lat/long
TB_merged = TB_merged.join(Tburoughs_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

TB_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

#set colors scheme for clusters
x = np.arange(kclusters)
ys = [i + x +(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers
markers_colors = []
for lat, lon, poi, cluster in zip(TB_merged['Latitude'], TB_merged['Longitude'], TB_merged['Neighborhood'], TB_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],
                       radius=5, 
                       popup=label, 
                       color=rainbow[cluster-1], 
                       fill=True, 
                       fill_color=rainbow[cluster-1], 
                       fill_opacity=0.7).add_to(map_clusters)
    
map_clusters